In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *
import requests

In [2]:
subway = pd.read_csv('./data/지하철위도경도(신).csv')
subway.head()

,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표
0,4128,삼성중앙,9호선(연장),127.053282,37.513011
1,4124,사평,9호선,127.015259,37.504206
2,4121,구반포,9호선,126.987332,37.501364
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770
4,4116,샛강,9호선,126.928422,37.517274


In [3]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 정보가 있는 데이터프레임 (house_df)에서 반복적으로 역 지오코딩을 수행합니다.
for index, row in subway.iterrows():
    latitude = row['환승역Y좌표']
    longitude = row['환승역X좌표']
    if latitude is not None and longitude is not None:
        url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode'
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get('results'):
                gu = data['results'][0]['region']['area2']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                dong = data['results'][0]['region']['area3']['name']  # 예시로 주소 중 'area3'의 'name' 사용
                subway.at[index, '법정동'] = dong  # 주소 정보를 데이터프레임에 추가
                subway.at[index, '시군구'] = gu  # 주소 정보를 데이터프레임에 추가
                
subway

,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표,법정동,시군구
0,4128,삼성중앙,9호선(연장),127.053282,37.513011,삼성동,강남구
1,4124,사평,9호선,127.015259,37.504206,반포동,서초구
2,4121,구반포,9호선,126.987332,37.501364,반포동,서초구
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770,흑석동,동작구
4,4116,샛강,9호선,126.928422,37.517274,여의도동,영등포구
...,...,...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.945064,37.469102,신림동,관악구
761,4306,신논현,신분당선(연장2),127.025060,37.504598,역삼동,강남구
762,1404,탕정,장항선,127.084850,36.788660,탕정면,아산시
763,4304,신사,신분당선(연장2),127.020114,37.516334,논현동,강남구


In [4]:
subway.columns

Index(['외구간_역_수', '역한글명칭', '호선명칭', '환승역X좌표', '환승역Y좌표', '법정동', '시군구'], dtype='object')

In [5]:
subway.drop(columns=['외구간_역_수', '호선명칭', '환승역X좌표', '환승역Y좌표'], inplace=True)
subway

,역한글명칭,법정동,시군구
0,삼성중앙,삼성동,강남구
1,사평,반포동,서초구
2,구반포,반포동,서초구
3,흑석(중앙대입구),흑석동,동작구
4,샛강,여의도동,영등포구
...,...,...,...
760,관악산(서울대),신림동,관악구
761,신논현,역삼동,강남구
762,탕정,탕정면,아산시
763,신사,논현동,강남구


In [6]:
subway_dongbuk = subway[(subway['시군구'] == '강북구') | (subway['시군구'] == '도봉구') | (subway['시군구'] == '노원구')| (subway['시군구'] == '중랑구')
                            | (subway['시군구'] == '성북구')| (subway['시군구'] == '동대문구')]
subway_dongbuk

,역한글명칭,법정동,시군구
10,청량리(서울시립대입구),전농동,동대문구
15,한성대입구(삼선교),삼선동1가,성북구
20,수유(강북구청),번동,강북구
26,제기동,용두동,동대문구
47,망우,상봉동,중랑구
...,...,...,...
667,용두(동대문구청),용두동,동대문구
670,당고개,상계동,노원구
680,도봉,도봉동,도봉구
681,회기,휘경동,동대문구


In [17]:
subway_dongbuk.to_csv('./data/지하철주소테스트.csv')
subway_dongbuk

,역한글명칭,법정동,시군구
10,청량리(서울시립대입구),전농동,동대문구
15,한성대입구(삼선교),삼선동1가,성북구
20,수유(강북구청),번동,강북구
26,제기동,용두동,동대문구
47,망우,상봉동,중랑구
...,...,...,...
667,용두(동대문구청),용두동,동대문구
670,당고개,상계동,노원구
680,도봉,도봉동,도봉구
681,회기,휘경동,동대문구


In [19]:
subway_dongbuk = pd.read_csv('./data/지하철주소테스트.csv', encoding='EUC-KR')
subway_dongbuk

,Unnamed: 0,역한글명칭,법정동,시군구
0,10,청량리(서울시립대입구),전농동,동대문구
1,15,한성대입구(삼선교),삼선동1가,성북구
2,20,수유(강북구청),번동,강북구
3,26,제기동,용두동,동대문구
4,47,망우,상봉동,중랑구
...,...,...,...,...
66,667,용두(동대문구청),용두동,동대문구
67,670,당고개,상계동,노원구
68,680,도봉,도봉동,도봉구
69,681,회기,휘경동,동대문구


In [20]:
# '시군구명'과 '법정동명'을 기준으로 그룹화하고 '법정동'의 데이터 수를 카운트
grouped = subway_dongbuk.groupby(['시군구', '법정동']).size().reset_index(name='역수')
grouped

,시군구,법정동,역수
0,강북구,미아동,5
1,강북구,번동,1
2,강북구,수유동,2
3,강북구,우이동,3
4,노원구,공릉동,4
5,노원구,상계동,6
6,노원구,월계동,2
7,노원구,중계동,1
8,노원구,하계동,1
9,도봉구,도봉동,3


In [21]:
grouped.to_csv('./data/동북부지하철수(신).csv')
grouped

,시군구,법정동,역수
0,강북구,미아동,5
1,강북구,번동,1
2,강북구,수유동,2
3,강북구,우이동,3
4,노원구,공릉동,4
5,노원구,상계동,6
6,노원구,월계동,2
7,노원구,중계동,1
8,노원구,하계동,1
9,도봉구,도봉동,3


In [22]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,강북구,미아동,4370.0
1,강북구,번동,2710.0
2,강북구,수유동,6240.0
3,강북구,우이동,10280.0
4,노원구,공릉동,8230.0
5,노원구,상계동,15680.0
6,노원구,월계동,4270.0
7,노원구,중계동,5200.0
8,노원구,하계동,2060.0
9,도봉구,도봉동,9550.0


In [23]:
width = pd.merge(width, grouped, on=['시군구', '법정동'], how='left')
width

,시군구,법정동,면적,역수
0,강북구,미아동,4370.0,5.0
1,강북구,번동,2710.0,1.0
2,강북구,수유동,6240.0,2.0
3,강북구,우이동,10280.0,3.0
4,노원구,공릉동,8230.0,4.0
5,노원구,상계동,15680.0,6.0
6,노원구,월계동,4270.0,2.0
7,노원구,중계동,5200.0,1.0
8,노원구,하계동,2060.0,1.0
9,도봉구,도봉동,9550.0,3.0


In [24]:
width['역수'].fillna(0, inplace=True)
width

,시군구,법정동,면적,역수
0,강북구,미아동,4370.0,5.0
1,강북구,번동,2710.0,1.0
2,강북구,수유동,6240.0,2.0
3,강북구,우이동,10280.0,3.0
4,노원구,공릉동,8230.0,4.0
5,노원구,상계동,15680.0,6.0
6,노원구,월계동,4270.0,2.0
7,노원구,중계동,5200.0,1.0
8,노원구,하계동,2060.0,1.0
9,도봉구,도봉동,9550.0,3.0


In [25]:
width['면적(㎢)당 역 수'] = width['역수'] / (width['면적'] * 0.001)
width

,시군구,법정동,면적,역수,면적(㎢)당 역 수
0,강북구,미아동,4370.0,5.0,1.144165
1,강북구,번동,2710.0,1.0,0.369004
2,강북구,수유동,6240.0,2.0,0.320513
3,강북구,우이동,10280.0,3.0,0.291829
4,노원구,공릉동,8230.0,4.0,0.486027
5,노원구,상계동,15680.0,6.0,0.382653
6,노원구,월계동,4270.0,2.0,0.468384
7,노원구,중계동,5200.0,1.0,0.192308
8,노원구,하계동,2060.0,1.0,0.485437
9,도봉구,도봉동,9550.0,3.0,0.314136


In [26]:
width.drop(columns=['면적', '역수'], inplace=True)
width

,시군구,법정동,면적(㎢)당 역 수
0,강북구,미아동,1.144165
1,강북구,번동,0.369004
2,강북구,수유동,0.320513
3,강북구,우이동,0.291829
4,노원구,공릉동,0.486027
5,노원구,상계동,0.382653
6,노원구,월계동,0.468384
7,노원구,중계동,0.192308
8,노원구,하계동,0.485437
9,도봉구,도봉동,0.314136


In [27]:
width.to_csv('./data/동북부지하철수(신).csv')
width

,시군구,법정동,면적(㎢)당 역 수
0,강북구,미아동,1.144165
1,강북구,번동,0.369004
2,강북구,수유동,0.320513
3,강북구,우이동,0.291829
4,노원구,공릉동,0.486027
5,노원구,상계동,0.382653
6,노원구,월계동,0.468384
7,노원구,중계동,0.192308
8,노원구,하계동,0.485437
9,도봉구,도봉동,0.314136
